In [137]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

In [138]:
class Estagio:

  def __init__(self, armazenamentos, afluencias, carga, custo_otimo):

    self.armazenamentos = armazenamentos
    self.afluencias = afluencias
    self.carga = carga

    self.results = {}
    self.custo_otimo = custo_otimo
  
  def interpolate_function(self, N_estagio):

      if N_estagio == 3:
      
        list_custo_otimo = np.zeros(len(self.armazenamentos))
      
      else:
        list_custo_otimo = []
        for key, value in self.custo_otimo.items():
          list_custo_otimo.append(value)

      z = np.polyfit(self.armazenamentos, list_custo_otimo, 2)

      #p = np.poly1d(z)

      #y_test = p(self.armazenamentos)

      #import matplotlib.pyplot as plt
      
      #_ = plt.plot(self.armazenamentos, y_test)
      #plt.show()

      return z

  def calculate_custo_imediato(self, gt1, gt2, defict):

    return 10 * gt1 + 25 * gt2 + 500 * defict

  def calculate_custo_futuro(self, ea):

      p = np.poly1d(self.polym)

      return p(ea)

  def format_results(self, results):

    x = results['x']
    custo_imediato = self.calculate_custo_imediato(x[0], x[1], x[2])
    custo_futuro = self.calculate_custo_futuro(x[4])

    variables = {}
    
    variables["gt1"] = round(x[0], 4)
    variables["gt2"] = round(x[1], 4)
    variables["deficit"] = round(x[2], 4)
    variables["ghidrs"] = round(x[3], 4)
    variables["ea"] = round(x[4], 4)
    variables["evert"] = round(x[5], 4)
    variables["custo-imediato"] = round(custo_imediato, 4)
    variables["custo-futuro"] = round(custo_futuro, 4)

    return variables

  def minimize(self, N_estagio):

    self.polym = self.interpolate_function(N_estagio)
    
    for EA in self.armazenamentos:
      for EC in self.afluencias:

        ### X = ["gt1", "gt2", "deficit", "ghidrs", "ea", "evert"]
        function = lambda x: 10 * x[0] + 25 * x[1] + 500 * x[2] + 0.01 * x[5] \
                          + self.polym[0] * pow((EA + EC - x[3] - x[5]), 2) \
                          + self.polym[1] * (EA + EC - x[3] - x[5]) + self.polym[2]

        constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] + x[3] - self.carga},
                       {'type': 'eq', 'fun': lambda x: x[3] + x[4] + x[5] - EA - EC})
        
        limits = ((0, 15), (0, 10), (0, None), (0, 76), (100, 160), (0, None))

        res = minimize(function, (1, 1, 1, 1, 1, 1), method='SLSQP', bounds=limits, constraints=constraints)

        self.results["EA"+str(EA)+"-EC"+str(EC)] = self.format_results(res)

    return self.results

  def calculate_custo_otimo(self):

    custo_otimo = {}
    for ea in self.armazenamentos:
      custo_ot = 0.0

      for ec in self.afluencias:
        
        custo_ot += (1/2) * self.results["EA"+str(ea)+"-EC"+str(ec)]['custo-imediato'] + (1/2) * self.results["EA"+str(ea)+"-EC"+str(ec)]['custo-futuro']

      custo_otimo[ea] = custo_ot

    return custo_otimo  


In [139]:
dict_vars = {3 : {"armazenamentos" : [160, 130, 100], "afluencias" : [42, 32], "carga":65},
             2:  {"armazenamentos" : [160, 130, 100], "afluencias" : [30, 18], "carga":65},
             1:  {"armazenamentos" : [160, 130, 100], "afluencias" : [26, 16], "carga":65}}

estagios = {}
for N_estagio, cenario in dict_vars.items():

  if N_estagio == 3:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], 0)

  else:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], estagios["custo_otimo-"+str(N_estagio + 1)])

  estagios["results-"+str(N_estagio)] = estagios[N_estagio].minimize(N_estagio)

  estagios["custo_otimo-"+str(N_estagio)] = estagios[N_estagio].calculate_custo_otimo()
  

In [140]:
table_3 = pd.DataFrame(data=estagios["results-3"])
table_2 = pd.DataFrame(data=estagios["results-2"])
table_1 = pd.DataFrame(data=estagios["results-1"])

In [141]:
table_3

,EA160-EC42,EA160-EC32,EA130-EC42,EA130-EC32,EA100-EC42,EA100-EC32
gt1,0.0,0.0,0.0,3.0,15.0,15.0
gt2,0.0,0.0,0.0,0.0,8.0,10.0
deficit,0.0,0.0,0.0,0.0,0.0,8.0
ghidrs,65.0,65.0,65.0,62.0,42.0,32.0
ea,137.0,127.0,107.0,100.0,100.0,100.0
evert,0.0,0.0,0.0,0.0,0.0,0.0
custo-imediato,0.0,0.0,0.0,30.0,350.0,4400.0
custo-futuro,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
table_2

,EA160-EC30,EA160-EC18,EA130-EC30,EA130-EC18,EA100-EC30,EA100-EC18
gt1,15.0000,15.0000,15.0000,15.0000,15.0,15.0
gt2,0.0000,7.5970,10.0000,10.0000,10.0,10.0
deficit,0.0000,0.0000,0.0000,0.0000,10.0,22.0
ghidrs,50.0000,42.4030,40.0000,40.0000,30.0,18.0
ea,140.0000,135.5970,120.0000,108.0000,100.0,100.0
evert,0.0000,0.0000,0.0000,0.0000,0.0,0.0
custo-imediato,150.0000,339.9252,400.0000,400.0000,5400.0,11400.0
custo-futuro,-250.5556,-165.7368,541.1111,1516.3778,2375.0,2375.0


In [143]:
table_1

,EA160-EC26,EA160-EC16,EA130-EC26,EA130-EC16,EA100-EC26,EA100-EC16
gt1,15.000,15.0000,15.0000,15.0000,15.0,15.0
gt2,10.000,10.0000,10.0000,10.0000,10.0,10.0
deficit,0.000,0.0000,0.0000,0.0000,14.0,24.0
ghidrs,40.000,40.0000,40.0000,40.0000,26.0,16.0
ea,146.000,136.0000,116.0000,106.0000,100.0,100.0
evert,0.000,0.0000,0.0000,0.0000,0.0,0.0
custo-imediato,400.000,400.0000,400.0000,400.0000,7400.0,12400.0
custo-futuro,-303.489,514.0126,4800.4585,8269.4027,10775.0,10775.0
